In [ ]:
pip install bubblebox

In [ ]:
# for optimal performance, open the Notebook environment using
# > jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

# More info on https://jupyter-notebook.readthedocs.io/en/stable/comms.html

In [ ]:
import bubblebox as bb
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

colors = interp1d(np.linspace(0,1,6), np.random.uniform(0,1,((3,6))))


%matplotlib notebook

In [ ]:
# original implementation of live visuals
# b = bb.mdbox(100, size = (5,5))
# b.run()

In [ ]:
from IPython import display

orbitcontrols = open("OrbitControls.js").read()

output_cell = """
        
        element.append("<div id='cell_output'></div>");
        require.config({paths: {three: "https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three"}});
        require(["three"], function(THREE) {
            // load external modules 
            %s
            
			const scene = new THREE.Scene();
            
			const camera = new THREE.PerspectiveCamera( 75, window.innerWidth / window.innerHeight, 0.1, 1000 );
            
            camera.position.z = 5;
            
            const light = new THREE.AmbientLight( 0x202020, 5 ); // soft white light
            scene.add( light );

			const directionalLight = new THREE.DirectionalLight( 0xffffff, .5 );
			directionalLight.position.set( 1, 1, 1 );
			//light.castShadow = true; // default false
            scene.add( directionalLight );
            
            


			const renderer = new THREE.WebGLRenderer();
			renderer.setSize( .5*window.innerWidth, .5*window.innerHeight );
            renderer.setClearColor( 0xf3efdf, 1);
            //renderer.antialias = true;
			
            // make renderer visible on cell output
            document.getElementById("cell_output").append(renderer.domElement)
            
            
            
            // function to initialize spheres from Python side
            var comm_manager=Jupyter.notebook.kernel.comm_manager
            var handle_init=function(msg){    
                
                for (let i =0; i < msg.content.data.length; i++) {
                    let geometry = new THREE.SphereGeometry(.04*msg.content.data[ i ][ 3 ]);
                    //let material = new THREE.MeshBasicMaterial( { color: 0x003320 } );
                    //let colormat = ;
                    let material = new THREE.MeshStandardMaterial( { color: "rgba(" + [msg.content.data[ i ][ 4 ], msg.content.data[ i ][ 5 ], msg.content.data[ i ][ 6 ]].join(",") + ")" } );
                    material.depthWrite = true;
                    material.castShadow = true;
                    material.shadowMapEnabled = true;
                    material.roughness = 0;
                    material.metalness = 0;
                    material.shininess = 1.0;
                    
                    
                    //let material = new THREE.MeshBasicMaterial( { color: "rgba(" + [msg.content.data[ i ][ 4 ], msg.content.data[ i ][ 4 ], msg.content.data[ i ][ 5 ], 0].join(",") + ")" } );
                    
                    let new_cube = new THREE.Mesh( geometry, material );
                    
                    new_cube.position.x = msg.content.data[ i ][ 0 ];
                    new_cube.position.y = msg.content.data[ i ][ 1 ];
                    new_cube.position.z = msg.content.data[ i ][ 2 ];
                    
                    scene.add( new_cube );
                }
                
            }

            comm_manager.register_target('initializeSpheres', function(comm,msg){
                // register callback
                comm.on_msg(handle_init)
            })
            
            
            
            
            
            
            // function to handle position update from Python side
            var handle_update=function(msg){    
                
                for (let i =0; i < msg.content.data.length; i++) {
                    scene.children[ i+2 ].position.x = msg.content.data[ i ][ 0 ];
                    scene.children[ i+2 ].position.y = msg.content.data[ i ][ 1 ];
                    scene.children[ i+2 ].position.z = msg.content.data[ i ][ 2 ];
                }
            }

            comm_manager.register_target('updatePositions', function(comm,msg){
                // register callback
                comm.on_msg(handle_update)
            })
            

			
            //let controls = new THREE.OrbitControls( camera, renderer.domElement );
            const controls = new THREE.OrbitControls( camera, renderer.domElement );

			function animate() {
				requestAnimationFrame( animate );
                //camera.rotation.x += .01;
                

				renderer.render( scene, camera );
			};

			animate();
            renderer.setAnimationLoop(null);
        });
		""" % orbitcontrols

display.Javascript(output_cell) #remove(renderer) 

In [ ]:
from ipykernel.comm import Comm 

b = bb.mdbox(500, size = (-9,-9,-9), vel = .1)
ci=Comm(target_name='initializeSpheres',data={})
cp=Comm(target_name='updatePositions',data={})

def initialize_spheres(b):
    init_array = np.zeros((7, b.pos.shape[1]))
    #print(init_array.shape, b.pos.shape)
    init_array[ :b.ndim, :]=b.pos 
    init_array[3, :]=b.masses
    col =np.array(255*colors(b.masses/b.masses.max()), dtype = int)
    #for i in range(b.pos.shape[1]):
        
    init_array[4:,:] = col #np.random.randint(0,255,(3, b.pos.shape[1]))
    ci.send((init_array.T).tolist())

b.interactions[:,:,1] *= 2
#b.vel_ = .6*np.roll(b.pos, 1, axis = 0)
#b.vel_[1] *= -1
#b.vel_ = b.vel_*np.exp(-.1*np.sum(b.pos**2, axis = 0)[None, :]**.5)
#b.vel_[0,:] += 10*np.exp(-.2*b.pos[1,:]**2)
b.masses = np.random.randint(1,15,b.pos.shape[1])
initialize_spheres(b)



while True:
    for j in range(2):
        b.advance()

    cp.send((b.pos.T).tolist())